<a href="https://colab.research.google.com/github/nalgo-intern/team-b/blob/master/%E7%99%BA%E8%A1%A8%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#driveからデータを取得

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#先にGoogle Driveで https://drive.google.com/drive/folders/1DxqM2rP85xaSXRiRfdnLjIJgA-4_U112?usp=sharing をマイドライブにコピーしておくこと 
!cp -R /content/drive/My\ Drive/learned_data* /content/

#サーバーを起動

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir uploads

--2019-08-30 01:11:21--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.245.79, 52.200.233.201, 52.3.53.115, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.245.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.9’

ngrok-stable-linux- 100%[===================>]  12.98M  74.5MB/s    in 0.2s    

2019-08-30 01:11:21 (74.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.9’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
mkdir: cannot create directory ‘uploads’: File exists


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
import os
import json
# request フォームから送信した情報を扱うためのモジュール
# redirect  ページの移動
# url_for アドレス遷移
from flask import Flask, request, redirect, url_for
# ファイル名をチェックする関数
from werkzeug.utils import secure_filename
# 画像のダウンロード
from flask import send_from_directory

from keras import models
import numpy as np
from keras.preprocessing import image
from keras.models import model_from_json
from keras import backend as K

# 画像のアップロード先のディレクトリ
UPLOAD_FOLDER = './uploads'
# アップロードされる拡張子の制限
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'gif'])

app = Flask(__name__)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


def gosanke_func(img_path):
    K.clear_session()
    gosanke =("フシギダネ", "ヒトカゲ", "ゼニガメ",
          "チコリータ", "ヒノアラシ", "ワニノコ",
          "キモリ", "アチャモ", "ミズゴロウ",
          "ナエトル", "ヒコザル", "ポッチャマ",
          "ツタージャ", "ポカブ", "ミジュマル",
          "ハリマロン", "フォッコ", "ケロマツ",
          "モクロー", "ニャビー", "アシマリ",
          "サルノリ", "ヒバニー", "メッソン")

    model = model_from_json(open('./learned_data_gosanke/data_gosanke.json').read())
    model.load_weights('./learned_data_gosanke/data_gosanke.hdf5')

    img_path = img_path 
    img = image.load_img(img_path, target_size=(120,120,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    features = model.predict(x)
    i = np.argmax(features[0])

    output = gosanke[i] 

    return output
    
def type_func(img_path):    
    K.clear_session()
    types = ("ノーマル", "ほのお", "みず", "くさ", "でんき", "こおり", "かくとう", "どく", "じめん", "ひこう", "エスパー", "むし", "いわ", "ゴースト", "ドラゴン")

    model = model_from_json(open('./learned_data/data.json').read())
    model.load_weights('./learned_data/data.hdf5')

    img_path = img_path 
    img = image.load_img(img_path, target_size=(120,120,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    features = model.predict(x)
    i = np.argmax(features[0])

    output = types[i] 
    
    return output
    
def result_func(filepath):
    gosanke_result = gosanke_func(filepath)
    type_result = type_func(filepath)
    return gosanke_result, type_result


def allwed_file(filename):
    # .があるかどうかのチェックと、拡張子の確認
    # OKなら１、だめなら0
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

    # ファイルを受け取る方法の指定
@app.route('/', methods=['GET', 'POST'])
def uploads_file():
    # リクエストがポストかどうかの判別
    if request.method == 'POST':
        # ファイルがなかった場合の処理
        if 'file' not in request.files:
            # flash('ファイルがありません')
            return redirect(request.url)
        # データの取り出し
        file = request.files['file']
        # ファイル名がなかった時の処理
        if file.filename == '':
            # flash('ファイルがありません')
            return redirect(request.url)
        # ファイルのチェック
        if file and allwed_file(file.filename):
            # 危険な文字を削除（サニタイズ処理）
            global filename
            filename = secure_filename(file.filename)
            # ファイルの保存
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            # アップロード後のページに転送
            print(file.filename)
            print(filename)
            
            # 関数呼び出し ポケモンの属性を文字で返す
            global target_path 
            target_path = "./uploads/" + filename

            global result_name
            global result_type
            result_name, result_type = result_func(target_path)
            
            return redirect(url_for('result'))

    return '''
    <!doctype html>
    <html>
        <head>
            <meta charset="UTF-8">
            <title>
                ポケモンタイプ診断
            </title>
            <link rel="stylesheet" href="./uploads/toppage.css">
        </head>

        <body> 
            <img src="./uploads/pokemon.jpg">
            <h1>
                ファイルをアップロードしてポケモンのタイプを判定しよう
            </h1>
            <ul>
                <li><b>好きな画像をアップロードしてみよう！</b></li>
                <li><b>その画像が何タイプなのかAIが教えてくれるよ</b></li>
                <li><b>タイプは全部で15種類！</b></li>
                <li><b>Let's Try!</b></li>
            </ul>
            <form method = post enctype = multipart/form-data>
            <p><input type=file class="q" name = file>
            <input type = submit class="qButton" value = 調べる>
            <div id="result"></div>
            </form>

            <script>
                const qs = (q) => document.querySelector(q)
                window.onload = () => {
                const q = qs('#q')
                const qButton = qs('#qButton')
                const result = qs('#result')
                // 判定ボタンを押した時 --- (*1)
                qButton.onclick = () => {
                    result.innerHTML = "..."
                    // APIサーバに送信するURLを構築 --- (*2)
                    const api = "/api?q=" + 
                    encodeURIComponent(q.value)
                    // APIにアクセス --- (*3)
                    fetch(api).then((res) => {
                    return res.json() // JSONで返す
                    }).then((data) => {
                    // 結果を画面に表示 --- (*4)
                    result.innerHTML =
                        data["label"] + 
                        "<span style='font-size:0.5em'>(" + 
                        data["per"] + ")</span>"
                    })
                }
                }
            </script>
        </body>
'''

@app.route('/uploads/<filename>')
# ファイルを表示する
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# resultした場合 
@app.route('/result', methods=['GET'])
def result():
    result_format = """
    <!DOCTYPE html>
    <html>
    <head><meta charset="UTF-8"><title>ポケモンタイプ</title>
    <link rel="stylesheet" href="./uploads/resultpage.css">
    </head>
    <body>
        <div align="center" class="main">
            <img src="{0}">
            
            <h1>結果画面</h1>
            <div align="center">
                <br>
                <h2>この画像は<font color="red">{2}</font>タイプです</h2>
                <br>
                <h2>この画像は<font color="red">{1}</font>に似てます</h2>
            </div>
            
            <b><a href="/" class="btn-gradient-3d-orange">もう一度調べる</a></b><br>
            
                <div id="social-platforms">
                <b><p>結果をSNSでシェアする</p></b>
                <a class="btn btn-icon btn-facebook" href="https://www.facebook.com/"><i class="fa fa-facebook"></i><span>Facebook</span></a>
                <a class="btn btn-icon btn-twitter" href="https://twitter.com/?lang=ja"><i class="fa fa-twitter"></i><span>Twitter</span></a>
                <a class="btn btn-icon btn-googleplus" href="https://plus.google.com/about?hl=ja"><i class="fa fa-google-plus"></i><span>Google+</span></a>
                <a class="btn btn-icon btn-pinterest" href="https://www.pinterest.jp/"><i class="fa fa-pinterest"></i><span>Pinterest</span></a>
                <a class="btn btn-icon btn-linkedin" href="https://www.linkedin.com/"><i class="fa fa-linkedin"></i><span>LinkedIn</span></a>
                </div>


        </div>
    </body>
    </html>
    """.format(target_path, result_name, result_type)
    return result_format 

if __name__ == '__main__':
    app.run(port=6006)

https://7da0aa35.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I0830 01:51:27.408204 140021820999552 _internal.py:122]  * Running on http://127.0.0.1:6006/ (Press CTRL+C to quit)
I0830 01:51:30.303525 140020087957248 _internal.py:122] 127.0.0.1 - - [30/Aug/2019 01:51:30] "GET / HTTP/1.1" 200 -
I0830 01:51:30.562394 140018733491968 _internal.py:122] 127.0.0.1 - - [30/Aug/2019 01:51:30] "GET /uploads/pokemon.jpg HTTP/1.1" 200 -
I0830 01:51:30.563572 140020087957248 _internal.py:122] 127.0.0.1 - - [30/Aug/2019 01:51:30] "GET /uploads/toppage.css HTTP/1.1" 200 -
I0830 01:51:30.879546 140020087957248 _internal.py:122] 127.0.0.1 - - [30/Aug/2019 01:51:30] "GET /favicon.ico HTTP/1.1" 404 -


18.jpg
18.jpg


E0830 01:51:35.514176 140020087957248 app.py:1891] Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-31-0a6f4ca5d388>", line 118, in uploads_file
    result_name, result_type = res

#バックアップ

In [0]:
!cp -R ./learned_data ./drive/My\ Drive
!cp dataset.npy ./drive/My\ Drive

In [0]:
!locale -a | grep ja